In [1]:
import pandas as pd

In [2]:
# Loading csv dataset
legitimate_urls = pd.read_csv("D:/downloads1/moodle6/Cybersecurity/Phishing-Website-Detection/extracted_csv_files/legitimate-urls.csv")
phishing_urls = pd.read_csv("D:/downloads1/moodle6/Cybersecurity/Phishing-Website-Detection/extracted_csv_files/phishing-urls.csv")

In [3]:
# Viewing both datasets
legitimate_urls.head(10)
phishing_urls.head(10)

Domain  Having_@_symbol  Having_IP  \
0                     asesoresvelfit.com                0          0   
1  caixa.com.br.fgtsagendesaqueconta.com                0          0   
2                      hissoulreason.com                0          0   
3              unauthorizd.newebpage.com                0          0   
4                         133.130.103.10                0          1   
5                             dj00.co.vu                1          0   
6                         133.130.103.10                0          1   
7                    httpssicredi.esy.es                0          0   
8                            gamesaty.ga                0          0   
9                   luxuryupgradepro.com                0          0   

                                              Path  Prefix_suffix_separation  \
0                           /media/datacredito.co/                         0   
1                   /consulta8523211/principal.php                         0   
2                               /js/homepage/home/                         0   
3                                  /webapps/66fbf/                         0   
4                                             /23/                         0   
5                                            /css/                         0   
6                                       /21/logar/                         0   
7  /servico/sicredi/validarclientes/mobi/index.php                         0   
8                             /wp-content///yh/en/                         0   
9                              /ymailNew/ymailNew/                         0   

  Protocol  Redirection_//_symbol  Sub_domains  URL_Length  age_domain  \
0     http                      0            0           0           0   
1     http                      0            1           1           0   
2     http                      0            0           0           0   
3     http                      0            0           0           0   
4     http                      0            2           0           1   
5     http                      0            0           2           1   
6     http                      0            2           0           1   
7     http                      0            2           2           1   
8     http                      1            0           2           1   
9     http                      0            0           0           0   

   dns_record  domain_registration_length  http_tokens  label  \
0           0                           1            0      1   
1           0                           1            0      1   
2           0                           1            0      1   
3           0                           1            0      1   
4           0                           1            0      1   
5           1                           1            0      1   
6           0                           1            0      1   
7           1                           1            1      1   
8           0                           1            0      1   
9           0                           1            0      1   

   statistical_report  tiny_url  web_traffic  
0                   0         1            1  
1                   1         0            1  
2                   0         0            1  
3                   1         0            1  
4                   0         0            1  
5                   1         0            0  
6                   0         0            1  
7                   1         0            1  
8                   0         0            1  
9                   0         0            1

## Data PreProcessing
#### Data is in two data frames so we merge them to make one dataframe
Note: two dataframes has same column names

In [4]:
urls = legitimate_urls.append(phishing_urls)

In [5]:
urls.head(5)

Domain  Having_@_symbol  Having_IP  \
0  www.liquidgeneration.com                0          0   
1       www.onlineanime.org                0          0   
2       www.ceres.dti.ne.jp                0          0   
3            www.galeon.com                0          0   
4       www.fanworkrecs.com                0          0   

                          Path  Prefix_suffix_separation Protocol  \
0                            /                         0     http   
1                            /                         0     http   
2  /~nekoi/senno/senfirst.html                         0     http   
3                        /kmh/                         0     http   
4                            /                         0     http   

   Redirection_//_symbol  Sub_domains  URL_Length  age_domain  dns_record  \
0                      0            0           0           0           0   
1                      0            0           0           0           0   
2                      0            1           0           1           0   
3                      0            0           0           0           0   
4                      0            0           0           1           1   

   domain_registration_length  http_tokens  label  statistical_report  \
0                           1            0      0                   0   
1                           1            0      0                   1   
2                           1            0      0                   0   
3                           0            0      0                   0   
4                           1            0      0                   1   

   tiny_url  web_traffic  
0         0            2  
1         0            1  
2         0            0  
3         0            0  
4         0            1

In [8]:
urls.shape

(2015, 17)

In [9]:
urls.columns

Index(['Domain', 'Having_@_symbol', 'Having_IP', 'Path',
       'Prefix_suffix_separation', 'Protocol', 'Redirection_//_symbol',
       'Sub_domains', 'URL_Length', 'age_domain', 'dns_record',
       'domain_registration_length', 'http_tokens', 'label',
       'statistical_report', 'tiny_url', 'web_traffic'],
      dtype='object')

#### Removing Unnecessary columns

In [10]:
urls = urls.drop(urls.columns[[0,3,5]],axis=1)

#### Since we merged two dataframes top 1000 rows will have legitimate urls and bottom 1000 rows will have phishing urls. So if we split the data now and create a model for it will overfit so we need to shuffle the rows before splitting the data into training set and test set

In [11]:
# randomly shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
urls = urls.sample(frac=1).reset_index(drop=True)

#### Removing class variable from the dataset

In [12]:
urls_without_labels = urls.drop('label',axis=1)
urls_without_labels.columns
labels = urls['label']


#### splitting the data into train data and test data

Dividing the data in the ratio of 70:30 [train_data:test_data]

In [13]:
from sklearn.model_selection import train_test_split
data_train, data_test, labels_train, labels_test = train_test_split(urls_without_labels, labels, test_size=0.30, random_state=110)

In [14]:
print(len(data_train),len(data_test),len(labels_train),len(labels_test))

1410 605 1410 605


#### checking the split of labels in train and test data

The split should be in equal proportion for both classes

Phishing - 1

Legitimate - 0


In [15]:
#initially checking the split of labels_train data 
labels_train.value_counts()


0    715
1    695
Name: label, dtype: int64

By above results it is shown that the split of both classes are almost equal!

Testing the same for labels of test data also

In [16]:
#checking the split for labels_test data
labels_test.value_counts()

1    303
0    302
Name: label, dtype: int64

as the split is almost in equal proportion we can train the model

#### Creating the model and fitting the data into the model

creating the model with default parameters

In [17]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier()

In [18]:
random_forest_classifier.fit(data_train,labels_train)


RandomForestClassifier()

#### Predicting the result for test data

In [19]:
prediction_label = random_forest_classifier.predict(data_test)

#### Creating confusion matrix and checking the accuracy

In [20]:
from sklearn.metrics import confusion_matrix,accuracy_score
cpnfusionMatrix = confusion_matrix(labels_test,prediction_label)
print(cpnfusionMatrix)
accuracy_score(labels_test,prediction_label)

[[260  42]
 [ 59 244]]


0.8330578512396695

### Improving the efficiency of model by specifying max_depth as well as number of tress 

In [21]:
custom_random_forest_classifier = RandomForestClassifier(n_estimators=500, max_depth=20, max_leaf_nodes=10000)

In [22]:
custom_random_forest_classifier.fit(data_train,labels_train)

RandomForestClassifier(max_depth=20, max_leaf_nodes=10000, n_estimators=500)

In [23]:
custom_classifier_prediction_label = custom_random_forest_classifier.predict(data_test)

In [24]:
#from sklearn.metrics import confusion_matrix,accuracy_score
confusionMatrix2 = confusion_matrix(labels_test,custom_classifier_prediction_label)
print(confusionMatrix2)
accuracy_score(labels_test,custom_classifier_prediction_label)

[[260  42]
 [ 58 245]]


0.8347107438016529